### Simple LLM App wiht LCEL

In this quickstart we’ll show you how to build a simple LLM application with LangChain.
This application will translate text from English into another language.
This is a relatively simple LLM application – it’s just a single LLM call plus some prompting.
Still, this is a great way to get started with LangChain – a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you’ll have a high-level overview of:
. Using language models

. Using PromptTemplates and OutputParsers

. Using LangChain Expression Language (LCEL) to chain components together

. Debugging and tracing your application using LangSmith

. Deploying app on Langserve

In [12]:
## OpenAI API Key and Opensource models
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [15]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F010F2E110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F010F2F580>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
## Messages to LLM 
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

response=model.invoke(messages)

In [19]:
response

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and casual greeting amongst friends.)\n* **Coucou, comment vas-tu ?** (A more familiar and friendly greeting, often used with close friends.)\n\n\n\nLet me know if you\'d like more variations!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 21, 'total_tokens': 130, 'completion_time': 0.198181818, 'prompt_time': 0.002142696, 'queue_time': 0.018349724, 'total_time': 0.200324514}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-652e379c-cb58-49a0-b8ec-b2b2783d88bb-0', usage_metadata={'input_tokens': 21, 'output_tokens': 109, 'total_tokens': 130})

In [23]:
## Parse the output to just get the response 
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(response)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and casual greeting amongst friends.)\n* **Coucou, comment vas-tu ?** (A more familiar and friendly greeting, often used with close friends.)\n\n\n\nLet me know if you\'d like more variations!\n'

In [24]:
## Using LCEL to chain components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual greeting, used with friends and family.)\n* **Bonjour, ça va ?** (This is a very common and informal greeting.)\n\n**Other options:**\n\n* **Coucou, comment ça va ?** (This is a very informal and friendly greeting, often used with close friends.)\n\nThe best option will depend on the context and your relationship with the person you are greeting.\n'

In [26]:
## Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}"

prompt=ChatPromptTemplate.from_messages([
    ("system",generic_template),("user","{text}")
])

In [30]:
result=prompt.invoke({"language":"Roman Urdu","text":"Hello,How are you"})

In [31]:
result.to_messages()

[SystemMessage(content='Translate the following into Roman Urdu', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello,How are you', additional_kwargs={}, response_metadata={})]

In [32]:
## Chaining using LCEL
chain=prompt|model|parser

chain.invoke({"language":"Roman Urdu","text":"Hello,How are you"})

'Here\'s how you\'d say "Hello, How are you?" in Roman Urdu:\n\n* **"Salam, kaise hain?"** \n\nLet me break it down:\n\n* **Salam:** This is the most common greeting in Urdu, meaning "Hello".\n* **kaise hain:** This means "How are you?" and is a polite way to ask someone about their well-being.\n\n\n\n'

In [1]:
!pip install fastapi

In [2]:
!pip install uvicorn

In [3]:
!pip install langserve

  Using cached langserve-0.3.1-py3-none-any.whl.metadata (40 kB)
Using cached langserve-0.3.1-py3-none-any.whl (1.2 MB)
